In [1]:
import pandas as pd 
import numpy as np 
import datetime as dt 
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r"./data.csv", encoding = 'unicode_escape')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [3]:
#check the shape (number of columns and rows) in the dataset
df.shape

(541909, 8)

In [4]:
df.isnull().sum(axis=0)

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [5]:
#formatting the time 
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [6]:
#dropping those without customerID
df = df[pd.notnull(df['CustomerID'])]

In [7]:
#cleaning the data without negative number of quantity
df = df[df['Quantity']>0]

In [8]:
#add column totalamount to calculate total spending per purchase
df['TotalAmount'] = df['Quantity'] * df['UnitPrice']
df2 = df[['CustomerID', 'InvoiceDate','InvoiceNo','Quantity', "TotalAmount"]]
df2.head()

,CustomerID,InvoiceDate,InvoiceNo,Quantity,TotalAmount
0,17850.0,2010-12-01 08:26:00,536365,6,15.30
1,17850.0,2010-12-01 08:26:00,536365,6,20.34
2,17850.0,2010-12-01 08:26:00,536365,8,22.00
3,17850.0,2010-12-01 08:26:00,536365,6,20.34
4,17850.0,2010-12-01 08:26:00,536365,6,20.34


In [9]:
#checking how many customer are left after we do some data cleaning
df2['CustomerID'].nunique()

4339

In [10]:
#check the last order date
df2.InvoiceDate.max()

Timestamp('2011-12-09 12:50:00')

In [11]:
#group customer ID and aggregate metrics

max_date = df2['InvoiceDate'].max()
df3 = df2.groupby('CustomerID').agg({'InvoiceDate': lambda x: (max_date -x.min()).days,
                                     'InvoiceNo': lambda x: x.nunique(),
                                     'Quantity':lambda x: x.sum(),
                                      'TotalAmount': lambda x: x.sum()})

df3.head()

,InvoiceDate,InvoiceNo,Quantity,TotalAmount
CustomerID,,,,
12346.0,325,1,74215,77183.60
12347.0,366,7,2458,4310.00
12348.0,357,4,2341,1797.24
12349.0,18,1,631,1757.55
12350.0,309,1,197,334.40


In [12]:
#change the name of the column 

df3.columns = ['Age', 'Frequency','Quantity','Monetary']
df3.head(10)


,Age,Frequency,Quantity,Monetary
CustomerID,,,,
12346.0,325,1,74215,77183.60
12347.0,366,7,2458,4310.00
12348.0,357,4,2341,1797.24
12349.0,18,1,631,1757.55
12350.0,309,1,197,334.40
12352.0,296,8,536,2506.04
12353.0,203,1,20,89.00
12354.0,231,1,530,1079.40
12355.0,213,1,240,459.40


In [13]:
#getting the element of CLTV calculation

df3['AOV'] = df3['Monetary']/df3['Frequency']
df3.head()

,Age,Frequency,Quantity,Monetary,AOV
CustomerID,,,,,
12346.0,325,1,74215,77183.60,77183.600000
12347.0,366,7,2458,4310.00,615.714286
12348.0,357,4,2341,1797.24,449.310000
12349.0,18,1,631,1757.55,1757.550000
12350.0,309,1,197,334.40,334.400000


In [14]:
#calculate the repeat purchase
pur_repeat = sum(df3['Frequency'])/len(df3)
pur_repeat

4.271952062687255

In [15]:
#calculate the repeat rate
repeat_rate = df3[df3['Frequency']>1].shape[0]/df3.shape[0]
repeat_rate

0.6556810324959668

In [16]:
#calculate the churn rate
churn_rate = 1 - repeat_rate
churn_rate

0.34431896750403324

In [17]:
#assume the profit margin
df3['ProfitMargin'] = df3['Monetary']*0.10

In [18]:
#putting equation together
df3['CLTV'] = df3['AOV']*pur_repeat/churn_rate*.10
df3.head(20)

,Age,Frequency,Quantity,Monetary,AOV,ProfitMargin,CLTV
CustomerID,,,,,,,
12346.0,325,1,74215,77183.60,77183.600000,7718.360,95761.392878
12347.0,366,7,2458,4310.00,615.714286,431.000,763.914324
12348.0,357,4,2341,1797.24,449.310000,179.724,557.457173
12349.0,18,1,631,1757.55,1757.550000,175.755,2180.585462
12350.0,309,1,197,334.40,334.400000,33.440,414.888782
12352.0,296,8,536,2506.04,313.255000,250.604,388.654262
12353.0,203,1,20,89.00,89.000000,8.900,110.421954
12354.0,231,1,530,1079.40,1079.400000,107.940,1339.207390
12355.0,213,1,240,459.40,459.400000,45.940,569.975797


In [19]:
print("The median CLTV is: {}".format(df3['CLTV'].median()))
print("The average CLTV is: {}".format(df3['CLTV'].mean()))

The median CLTV is: 364.3800428380187
The average CLTV is: 519.9108564852671


In [20]:
df3.sort_values(by='CLTV').reset_index()

,CustomerID,Age,Frequency,Quantity,Monetary,AOV,ProfitMargin,CLTV
0,13256.0,13,1,12540,0.00,0.000000,0.000,0.000000
1,16454.0,63,2,3,6.90,3.450000,0.690,4.280402
2,16738.0,297,1,3,3.75,3.750000,0.375,4.652610
3,14792.0,63,1,2,6.20,6.200000,0.620,7.692316
4,17816.0,364,4,30,36.56,9.140000,3.656,11.339963
...,...,...,...,...,...,...,...,...
4334,12357.0,32,1,2708,6207.67,6207.670000,620.767,7701.832070
4335,15098.0,181,3,121,39916.50,13305.500000,3991.650,16508.082195
4336,15749.0,331,3,18028,44534.30,14844.766667,4453.430,18417.844373
4337,12346.0,325,1,74215,77183.60,77183.600000,7718.360,95761.392878


Source: https://www.youtube.com/watch?v=nhD7yC9JpiA